# NGS Sequence Slim

In [1]:
import pandas as pd
import os

# Functions 

In [2]:
def drop_unnamed_columns(df): 
    return df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [3]:
def read_csv_with_na(filepath):
    df = pd.read_csv(filepath)
    return df.fillna('N/A')

In [4]:
# Introduce the file
filepath1 = 'NGS_Sequencing_Tracking_v2_2022-CURRENT.xlsx'
filepath2 = 'NGS_Sequence_Tracking_v1_2020-10Jan2022.xlsx'
filepath3 = 'RnaSeq_VENDOR_Tracking_Sheet_5_4_2020.xlsx'


In [5]:
# Define sheet names to combine
rna_seq_sheets = ['RNAseq_Template_Manifest', 'RNAseq_OLD_to_analyze']
wes_sheets = ['WES_Template_Manifest', 'WES_OLD_to_analyze']

In [6]:
# Read filepath1 to dictionary
xl = pd.ExcelFile(filepath1)
sheet_data = {sheet_name: xl.parse(sheet_name) for sheet_name in xl.sheet_names}

In [7]:
# Combine RNASEQ sheets
rna_seq_combined = pd.concat([drop_unnamed_columns(sheet_data[sheet]) for sheet in rna_seq_sheets], ignore_index=True)

In [8]:
# Combine WES sheets
wes_combined = pd.concat([drop_unnamed_columns(sheet_data[sheet]) for sheet in wes_sheets], ignore_index=True)

In [9]:
# Read filepath2 to dictionary
x2 = pd.ExcelFile(filepath2)
sheet_data2 = x2.parse('Sheet1')

In [10]:
# Split the data based on the 18th column
mask = sheet_data2.iloc[:, 16].str.startswith('RNA').fillna(False)
rna_seq_file2 = sheet_data2[mask]

mask = sheet_data2.iloc[:, 16].str.startswith('WES').fillna(False)
wes_file2 = sheet_data2[mask]

In [11]:
# wes_data = sheet_data2[sheet_data2['Panel'] == "SureSelect Human All Exon V7"] 
# Another way to split data based on NGS data column, (more specific and efficient)

In [12]:
# wes_data

In [13]:
# rna_data = sheet_data2[sheet_data2['Panel'] == ] 

In [14]:
# rna_data

In [15]:
x3 = pd.ExcelFile(filepath3)
sheet_data3 = x3.parse('RnaSeq_VENDOR_Tracking_Sheet_5_')

In [16]:
# filter 
rna_vendor = sheet_data3[sheet_data3['RNASeq.pipeline.status'] == 'Complete']

In [17]:
rna_vendor.to_csv('RNA_Vendor_V2.csv', index=False)

In [18]:
rna_seq_combined.to_csv('RNA_SEQ_Tracking_Current.csv', index=False)
wes_combined.to_csv('WES_Tracking_Current.csv', index=False)

In [19]:
rna_seq_file2.to_csv('RNA_SEQ_Tracking_Old.csv', index=False)
wes_file2.to_csv('WES_Tracking_Old.csv', index=False)

# Combining Old and New Tracking Sheets

In [20]:
# Define paths
rna_seq_current_path = 'RNA_SEQ_Tracking_Current.csv'
rna_seq_old_path = 'RNA_SEQ_Tracking_Old.csv'

In [21]:
wes_current_path = 'WES_Tracking_Current.csv'
wes_old_path = 'WES_Tracking_Old.csv'

In [22]:
rna_seq_current_df = read_csv_with_na(rna_seq_current_path)
rna_seq_old_df = read_csv_with_na(rna_seq_old_path)

In [23]:
wes_current_df = read_csv_with_na(wes_current_path)
wes_old_df = read_csv_with_na(wes_old_path)

In [24]:
# Combine df w/ alike headers
rna_seq_combined = pd.concat([rna_seq_current_df, rna_seq_old_df, rna_vendor], ignore_index=True, sort=False)

In [25]:
wes_combined = pd.concat([wes_current_df, wes_old_df], ignore_index=True, sort=False)

In [26]:
rna_seq_combined['Sample_Name'] = rna_seq_combined['Sample_Name'].combine_first(rna_seq_combined['Model'])

# drop column
rna_seq_combined.drop(columns=['Model'], inplace=True)

In [27]:
# Combining WES Headers

In [28]:
wes_combined['Sample_Name'] = wes_combined['Sample_Name'].combine_first(wes_combined['Model'])

# drop column
wes_combined.drop(columns=['Model'], inplace=True)

In [30]:
rna_seq_combined = rna_seq_combined.fillna('N/A')
wes_combined = wes_combined.fillna('N/A')

# Save 
rna_seq_combined.to_csv('RNA_SEQ_Final.csv', index=False)
wes_combined.to_csv('WES_Final.csv', index=False)